In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN as GCN_DR
from deeprobust.graph.global_attack import DICE
from scipy.sparse import csr_matrix

In [2]:
from greedy_mcmc_attack import *
from collections import defaultdict

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
cora_dataset = Planetoid(root='', name='Pubmed')
data = cora_dataset[0].to(device)
print(data)
idx_train, idx_val, idx_test = data.train_mask.nonzero(as_tuple=True)[0].tolist(), data.val_mask.nonzero(as_tuple=True)[0].tolist(), data.test_mask.nonzero(as_tuple=True)[0].tolist()

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [4]:
data_dr = Dataset(root='/tmp/', name='pubmed', setting='nettack')
adj, features, labels = data_dr.adj, data_dr.features, data_dr.labels
# idx_train, idx_val, idx_test = data_dr.idx_train, data_dr.idx_val, data_dr.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading pubmed dataset...


In [5]:
surrogate = GCN_DR(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [6]:
adj = csr_matrix(adj)

In [7]:
ptb_rate = 0.1

In [8]:
model = DICE()
model = model.to(device)
perturbations = int(ptb_rate * (adj.sum() // 2))
model.attack(adj, labels, n_perturbations=perturbations)
modified_adj = model.modified_adj

number of pertubations: 4432


### arch tests

In [19]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [20]:
model_save_path = "../models/pubmed_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges.pth" # pubmed doesn't exist yet... have to satiate

In [21]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [22]:
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.784
Initial Loss: 0.5719911456108093


In [23]:
from scipy.sparse import csr_matrix

adj1 = data_dr.adj.tocsr()
adj2 = modified_adj.tocsr()

arr_diff = (adj1 - adj2).tocoo()

In [24]:
arr_diff

<19717x19717 sparse array of type '<class 'numpy.int64'>'
	with 8863 stored elements in COOrdinate format>

In [25]:
data_modded = copy.deepcopy(data)

In [26]:
# thanks gpt
existing = data_modded.edge_index.t().cpu().numpy()

diff_coo = arr_diff.tocoo()
diff_edges = np.stack([diff_coo.row, diff_coo.col], axis=1)

existing_set = set(map(tuple, existing))
diff_set = set(map(tuple, diff_edges))
diff_set_full = diff_set | set((v, u) for u, v in diff_set)

cleaned_edges = existing_set - diff_set_full
print(len(cleaned_edges))

added_edges = diff_set_full - existing_set
print(len(added_edges))

final_edges = cleaned_edges | added_edges

final_edges = torch.tensor(list(final_edges), dtype=torch.long).t().to(device)
data_modded.edge_index = final_edges

84210
4425


In [27]:
data_modded.edge_index.size()

torch.Size([2, 88635])

In [28]:
new_loss, new_accuracy = train.test(data_modded)

print(f"Evasion Change in Accuracy: {(100 * ((new_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Evasion Change in Loss: {(100 * ((new_loss - initial_loss) / initial_loss)):.4f}%")

### poisoning
model_poisoning = GCN(data_modded.x.shape[1], cora_dataset.num_classes, [16]).to(device)
model_poisoning.reset_parameters()
train_modded = Trainable(model_poisoning)
train_modded.fit(data_modded, 1000)

poisoning_loss, poisoning_accuracy = train_modded.test(data_modded)
print(f"Poisoning Change in Accuracy: {(100 * ((poisoning_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Poisoning Change in Loss: {(100 * ((poisoning_loss - initial_loss) / initial_loss)):.4f}%")

Evasion Change in Accuracy: -2.6786%
Evasion Change in Loss: 7.0909%
Epoch 0, Train Loss - 1.098446011543274, Val Loss - 1.0931408405303955, Val Accuracy - 0.554
Epoch 20, Train Loss - 0.7190119028091431, Val Loss - 0.8770945072174072, Val Accuracy - 0.744
Epoch 40, Train Loss - 0.35322800278663635, Val Loss - 0.6844544410705566, Val Accuracy - 0.756
Epoch 60, Train Loss - 0.22162680327892303, Val Loss - 0.6194964051246643, Val Accuracy - 0.766
Epoch 80, Train Loss - 0.1710447520017624, Val Loss - 0.6164628863334656, Val Accuracy - 0.762
Epoch 100, Train Loss - 0.13820190727710724, Val Loss - 0.619831919670105, Val Accuracy - 0.76
Epoch 120, Train Loss - 0.12509037554264069, Val Loss - 0.6089637875556946, Val Accuracy - 0.758
Epoch 140, Train Loss - 0.10900295525789261, Val Loss - 0.6102266311645508, Val Accuracy - 0.752
Epoch 160, Train Loss - 0.08434725552797318, Val Loss - 0.6131449937820435, Val Accuracy - 0.754
Epoch 180, Train Loss - 0.08533189445734024, Val Loss - 0.612061083316